In [1]:
# Import packages
import pandas as pd 
import numpy as np 
import geopandas as gpd 
import matplotlib.pyplot as plt

from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import io
import json

import plotly.express as px
from collections import Counter
import glob


In [7]:
df = pd.read_csv("../data/raw/city_jan_2020/full_city_jan_2020.csv")
df['recording_time'] = pd.to_datetime(df['recording_time'], format="%Y-%m-%d %H:%M:%S")

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'],df['lat'], crs="EPSG:4326"))
gdf = gdf.to_crs("EPSG:28992")


In [8]:
gdf["date"] = gdf['recording_time'].dt.date
gdf["hour"] = gdf["recording_time"].dt.hour
gdf["datehour"] = pd.to_datetime(gdf["date"]) + gdf["hour"].astype('timedelta64[h]')

count = Counter([date for date in gdf["datehour"]])

df = pd.DataFrame.from_dict(count, orient='index', columns=["activity"]).reset_index().sort_values(by=['index'])

fig = px.line(df, x='index', y='activity', title='Snuffelfiets activity per hour in January 2020')
fig.show()

In [10]:
mean_pm25_date = gdf[["date", "pm2_5"]].groupby(["date"]).median()

fig = px.scatter(mean_pm25_date, x=mean_pm25_date.index, y='pm2_5', title='Median PM2.5 concentration per day (ug/m3)')
fig.show()

In [137]:
gdf['hour'] = gdf['recording_time'].dt.hour

mean_pm25_date = gdf[["date", "hour", "pm2_5"]].groupby(["date", "hour"]).median().reset_index()

mean_pm25_date["datetime"] = pd.to_datetime(mean_pm25_date["date"]) + mean_pm25_date["hour"].astype('timedelta64[h]')



fig = px.scatter(mean_pm25_date, x="datetime", y='pm2_5', title='Median PM2.5 concentration per hour in January (ug/m3)')
fig.show()

In [80]:
gdf.describe()

,air_quality_observed_id,lon,lat,trip_sequence,humidity,pm2_5,pressure,temperature,distance,delta_time,avg_speed_ms,hour
count,2.162390e+05,216239.000000,216239.000000,216239.000000,216239.000000,216239.000000,2.162390e+05,216239.000000,216239.000000,216239.000000,216239.000000,216239.000000
mean,1.845164e+07,5.104288,52.088833,8.008518,68.808379,11.594139,5.363640e+04,11.246530,95.060762,19.787309,5.148740,12.951244
std,7.543336e+06,0.037193,0.016843,7.342861,13.689889,11.236791,1.502583e+06,45.138878,1367.935385,354.682894,1.392429,4.585497
min,1.385415e+06,4.974094,52.028873,0.000000,0.000000,1.000000,4.310000e+02,0.000000,8.188689,2.000000,2.000223,0.000000
25%,1.103031e+07,5.084321,52.078094,3.000000,58.000000,4.000000,9.880000e+02,7.900000,55.776372,13.000000,4.200075,8.000000
50%,2.284533e+07,5.109617,52.089329,6.000000,71.000000,8.000000,1.004000e+03,10.200000,68.249257,13.000000,5.119263,13.000000
75%,2.351275e+07,5.129708,52.099194,11.000000,80.000000,16.000000,1.021000e+03,13.000000,82.233590,14.000000,6.144976,17.000000
max,2.420261e+07,5.195133,52.137924,59.000000,100.000000,150.000000,4.294950e+07,6553.500000,134791.032089,62375.000000,12.493945,23.000000


## "Weighted" K-means clustering of the PM2.5 data

### 1. Full data
### 2. Aggregated day/month medians

In [81]:
# Clustering
from sklearn import cluster
# Scaling
from sklearn.preprocessing import MinMaxScaler

In [90]:
full_data = gdf[["lon", "lat", "pm2_5", "geometry"]]

full_data['x'] = full_data["geometry"].apply(lambda p: p.x)
full_data['y'] = full_data["geometry"].apply(lambda p: p.y)

full_data.head()

C:\Users\Klant\anaconda3\envs\thesis\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Klant\anaconda3\envs\thesis\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,lon,lat,pm2_5,geometry,x,y
0,5.118134,52.111160,22,POINT (136568.331 458137.319),136568.331468,458137.319000
1,5.119091,52.110570,23,POINT (136633.645 458071.434),136633.644900,458071.433727
2,5.119953,52.110460,21,POINT (136692.649 458058.978),136692.648797,458058.977584
3,5.120988,52.109900,20,POINT (136763.320 457996.412),136763.320401,457996.412136
4,5.121899,52.109608,22,POINT (136825.608 457963.696),136825.608202,457963.696134


In [97]:
k_means = cluster.KMeans(n_clusters=20, random_state=42)
k_means.fit(full_data[["x","y","pm2_5"]])

X_cluster = k_means.labels_
full_data["cluster"] = X_cluster

C:\Users\Klant\anaconda3\envs\thesis\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [95]:
full_data.head(10)

,lon,lat,pm2_5,geometry,x,y,cluster
0,5.118134,52.111160,22,POINT (136568.331 458137.319),136568.331468,458137.319000,4
1,5.119091,52.110570,23,POINT (136633.645 458071.434),136633.644900,458071.433727,4
2,5.119953,52.110460,21,POINT (136692.649 458058.978),136692.648797,458058.977584,4
3,5.120988,52.109900,20,POINT (136763.320 457996.412),136763.320401,457996.412136,4
4,5.121899,52.109608,22,POINT (136825.608 457963.696),136825.608202,457963.696134,4
5,5.123208,52.109820,22,POINT (136915.338 457986.956),136915.337921,457986.956362,4
6,5.123628,52.110344,23,POINT (136944.348 458045.151),136944.348344,458045.151477,4
7,5.123058,52.109630,24,POINT (136905.033 457965.854),136905.033474,457965.854342,4
8,5.122562,52.108715,23,POINT (136870.664 457864.176),136870.664119,457864.176145,4
9,5.123335,52.108700,22,POINT (136923.640 457862.314),136923.639897,457862.314474,4


In [98]:
full_data.to_csv("test.csv", index=False)

In [99]:
k_means = cluster.KMeans(n_clusters=20, random_state=42)
k_means.fit(full_data[["x","y"]])

X_cluster = k_means.labels_
full_data["cluster"] = X_cluster

full_data.to_csv("test2.csv", index=False)

C:\Users\Klant\anaconda3\envs\thesis\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [113]:
x = full_data["x"].values.reshape(-1, 1)
y = full_data["y"].values.reshape(-1, 1)
pm = full_data["pm2_5"].values.reshape(-1, 1)

In [114]:
scaler = MinMaxScaler()
full_data["x_mm"] = scaler.fit_transform(x)
full_data["y_mm"] = scaler.fit_transform(y)
full_data["pm_mm"] = scaler.fit_transform(pm)

C:\Users\Klant\anaconda3\envs\thesis\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Klant\anaconda3\envs\thesis\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Klant\anaconda3\envs\thesis\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [115]:
k_means = cluster.KMeans(n_clusters=20, random_state=42)
k_means.fit(full_data[["x_mm","y_mm","pm_mm"]])

X_cluster = k_means.labels_
full_data["cluster"] = X_cluster
full_data.to_csv("test3.csv", index=False)

C:\Users\Klant\anaconda3\envs\thesis\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
df = pd.read_csv("../data/interim/district/daily_city_jan_2020.csv")
df['datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'],df['y'], crs="EPSG:28992"))
#gdf = gdf.to_crs("EPSG:28992")

In [10]:
df

,WK_NAAM,datetime,pm2_5,virtual_measurement_station,x,y,geometry
0,Wijk 01 West,2020-01-01 08:00:00,43.0,POINT (133565.4460767263 457411.0985499993),133565.446077,457411.09855,POINT (133565.446 457411.099)
1,Wijk 01 West,2020-01-02 08:00:00,15.0,POINT (133565.4460767263 457411.0985499993),133565.446077,457411.09855,POINT (133565.446 457411.099)
2,Wijk 01 West,2020-01-03 12:00:00,6.0,POINT (133565.4460767263 457411.0985499993),133565.446077,457411.09855,POINT (133565.446 457411.099)
3,Wijk 01 West,2020-01-04 13:00:00,3.0,POINT (133565.4460767263 457411.0985499993),133565.446077,457411.09855,POINT (133565.446 457411.099)
4,Wijk 01 West,2020-01-05 23:00:00,7.0,POINT (133565.4460767263 457411.0985499993),133565.446077,457411.09855,POINT (133565.446 457411.099)
...,...,...,...,...,...,...,...
303,Wijk 10 Vleuten-De Meern,2020-01-27 10:00:00,9.0,POINT (130281.9218950086 455160.5470000003),130281.921895,455160.54700,POINT (130281.922 455160.547)
304,Wijk 10 Vleuten-De Meern,2020-01-28 16:00:00,2.0,POINT (130281.9218950086 455160.5470000003),130281.921895,455160.54700,POINT (130281.922 455160.547)
305,Wijk 10 Vleuten-De Meern,2020-01-29 14:00:00,3.0,POINT (130281.9218950086 455160.5470000003),130281.921895,455160.54700,POINT (130281.922 455160.547)
306,Wijk 10 Vleuten-De Meern,2020-01-30 15:00:00,5.0,POINT (130281.9218950086 455160.5470000003),130281.921895,455160.54700,POINT (130281.922 455160.547)


In [11]:
gdf

,WK_NAAM,datetime,pm2_5,virtual_measurement_station,x,y,geometry
0,Wijk 01 West,2020-01-01 08:00:00,43.0,POINT (133565.4460767263 457411.0985499993),133565.446077,457411.09855,POINT (133565.446 457411.099)
1,Wijk 01 West,2020-01-02 08:00:00,15.0,POINT (133565.4460767263 457411.0985499993),133565.446077,457411.09855,POINT (133565.446 457411.099)
2,Wijk 01 West,2020-01-03 12:00:00,6.0,POINT (133565.4460767263 457411.0985499993),133565.446077,457411.09855,POINT (133565.446 457411.099)
3,Wijk 01 West,2020-01-04 13:00:00,3.0,POINT (133565.4460767263 457411.0985499993),133565.446077,457411.09855,POINT (133565.446 457411.099)
4,Wijk 01 West,2020-01-05 23:00:00,7.0,POINT (133565.4460767263 457411.0985499993),133565.446077,457411.09855,POINT (133565.446 457411.099)
...,...,...,...,...,...,...,...
303,Wijk 10 Vleuten-De Meern,2020-01-27 10:00:00,9.0,POINT (130281.9218950086 455160.5470000003),130281.921895,455160.54700,POINT (130281.922 455160.547)
304,Wijk 10 Vleuten-De Meern,2020-01-28 16:00:00,2.0,POINT (130281.9218950086 455160.5470000003),130281.921895,455160.54700,POINT (130281.922 455160.547)
305,Wijk 10 Vleuten-De Meern,2020-01-29 14:00:00,3.0,POINT (130281.9218950086 455160.5470000003),130281.921895,455160.54700,POINT (130281.922 455160.547)
306,Wijk 10 Vleuten-De Meern,2020-01-30 15:00:00,5.0,POINT (130281.9218950086 455160.5470000003),130281.921895,455160.54700,POINT (130281.922 455160.547)


In [15]:
df["weekday"] = df['datetime'].dt.dayofweek

In [25]:
df["weekday2"] = ((df["weekday"] == 1) | (df["weekday"] == 2) | (df["weekday"] == 3) | (df["weekday"] == 4) | (df["weekday"] == 5)).astype(int)

In [27]:
df.describe()

,pm2_5,x,y,weekday,weekday2
count,308.000000,308.000000,308.000000,308.000000,308.000000
mean,10.909091,135238.106780,455930.830119,2.980519,0.746753
std,11.181499,2357.394748,1972.551705,1.911633,0.435579
min,1.000000,130281.921895,452911.047050,0.000000,0.000000
25%,4.000000,133565.446077,454388.051000,1.000000,0.000000
50%,7.000000,135714.041522,455644.382800,3.000000,1.000000
75%,15.000000,137343.250783,457471.446000,5.000000,1.000000
max,124.000000,138258.201867,459241.864500,6.000000,1.000000


In [28]:
df.to_csv("features.csv")

In [35]:
df = pd.read_csv("../data/interim/neighborhood/daily_city_jan_2020.csv")
df['datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")

df["weekday"] = df['datetime'].dt.dayofweek
df['datetime'] = df['datetime'].dt.date

df["weekday2"] = ((df["weekday"] == 1) | (df["weekday"] == 2) | (df["weekday"] == 3) | (df["weekday"] == 4) | (df["weekday"] == 5)).astype(int)

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'],df['y'], crs="EPSG:28992"))

df.to_csv("features-neigh.csv")

In [36]:
gdf

,BU_NAAM,datetime,pm2_5,virtual_measurement_station,x,y,weekday,weekday2,geometry
0,2e Daalsebuurt en omgeving,2020-01-01,28.0,POINT (135343.7846732856 456782.7199999988),135343.784673,456782.7200,2,1,POINT (135343.785 456782.720)
1,2e Daalsebuurt en omgeving,2020-01-02,19.0,POINT (135343.7846732856 456782.7199999988),135343.784673,456782.7200,3,1,POINT (135343.785 456782.720)
2,2e Daalsebuurt en omgeving,2020-01-04,4.0,POINT (135343.7846732856 456782.7199999988),135343.784673,456782.7200,5,1,POINT (135343.785 456782.720)
3,2e Daalsebuurt en omgeving,2020-01-06,10.0,POINT (135343.7846732856 456782.7199999988),135343.784673,456782.7200,0,0,POINT (135343.785 456782.720)
4,2e Daalsebuurt en omgeving,2020-01-07,11.0,POINT (135343.7846732856 456782.7199999988),135343.784673,456782.7200,1,1,POINT (135343.785 456782.720)
...,...,...,...,...,...,...,...,...,...
3025,Zuilen-Noord,2020-01-27,10.0,POINT (133259.6657714601 459263.8550999984),133259.665771,459263.8551,0,0,POINT (133259.666 459263.855)
3026,Zuilen-Noord,2020-01-28,1.0,POINT (133259.6657714601 459263.8550999984),133259.665771,459263.8551,1,1,POINT (133259.666 459263.855)
3027,Zuilen-Noord,2020-01-29,3.0,POINT (133259.6657714601 459263.8550999984),133259.665771,459263.8551,2,1,POINT (133259.666 459263.855)
3028,Zuilen-Noord,2020-01-30,5.0,POINT (133259.6657714601 459263.8550999984),133259.665771,459263.8551,3,1,POINT (133259.666 459263.855)


In [26]:
df = pd.read_csv("../data/external/city_jan_2020/city_jan_2020.csv")
df['recording_time'] = pd.to_datetime(df['recording_time'], format="%Y-%m-%d %H:%M:%S")

df["week"] = df["recording_time"].dt.week
df["weekday"] = df["recording_time"].dt.dayofweek
df['hour'] = df['recording_time'].dt.hour

df = df.loc[((df["week"] == 2) | (df["week"] == 3))]
df = df.loc[((df["weekday"] >= 1) & (df["weekday"] <= 5))]
df = df.loc[~((df["hour"]>0) & (df["hour"]<7))]

<ipython-input-26-40cfd89c2b3e>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



In [23]:
df["hour"].unique()

array([ 8, 16, 17, 21,  9, 19, 11, 12, 13, 14, 18, 22, 23, 10, 15,  7,  0,
       20], dtype=int64)

In [24]:
df

,sensor,air_quality_observed_id,lon,lat,recording_time,trip_sequence,humidity,pm2_5,pressure,temperature,distance,delta_time,avg_speed_ms,week,weekday,hour
97350,00da8ce393d693c511be23980a82574e,11179270,5.192822,52.083153,2020-01-14 08:46:37,4,86,4,1014,11.1,55.110213,13.0,4.239247,3,1,8
97351,00da8ce393d693c511be23980a82574e,11179272,5.191824,52.083160,2020-01-14 08:46:50,4,86,5,1014,11.1,68.348859,13.0,5.257605,3,1,8
97352,00da8ce393d693c511be23980a82574e,11179273,5.190839,52.083149,2020-01-14 08:47:03,4,86,5,1014,11.1,67.571157,13.0,5.197781,3,1,8
97353,00da8ce393d693c511be23980a82574e,11179274,5.189839,52.083149,2020-01-14 08:47:16,4,86,4,1014,11.1,68.539709,13.0,5.272285,3,1,8
97354,00da8ce393d693c511be23980a82574e,11179271,5.188880,52.083183,2020-01-14 08:47:29,4,86,5,1014,11.0,65.807073,13.0,5.062083,3,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202779,fc3390e472769204c31cc9e0d83bf2bd,10728317,5.088515,52.062374,2020-01-09 18:10:03,3,85,7,1003,13.7,63.975855,14.0,4.569704,2,3,18
202780,fc3390e472769204c31cc9e0d83bf2bd,10728318,5.089257,52.062153,2020-01-09 18:10:16,3,85,7,1004,13.8,56.523031,13.0,4.347925,2,3,18
202781,fc3390e472769204c31cc9e0d83bf2bd,10728319,5.089505,52.061741,2020-01-09 18:10:29,3,85,6,1004,13.8,48.878257,13.0,3.759866,2,3,18
202782,fc3390e472769204c31cc9e0d83bf2bd,10728320,5.089443,52.061382,2020-01-09 18:10:43,3,85,4,1004,13.8,40.121249,14.0,2.865804,2,3,18


In [27]:
df['date'] = df["recording_time"].dt.date

df["date"].unique()

array([datetime.date(2020, 1, 14), datetime.date(2020, 1, 18),
       datetime.date(2020, 1, 15), datetime.date(2020, 1, 16),
       datetime.date(2020, 1, 17), datetime.date(2020, 1, 7),
       datetime.date(2020, 1, 8), datetime.date(2020, 1, 9),
       datetime.date(2020, 1, 11), datetime.date(2020, 1, 10)],
      dtype=object)

In [28]:
df["weekday"].unique()

array([1, 5, 2, 3, 4], dtype=int64)

In [30]:
df["recording_time"].dt.isocalendar().week

97350     3
97351     3
97352     3
97353     3
97354     3
         ..
202779    2
202780    2
202781    2
202782    2
202783    2
Name: week, Length: 75548, dtype: UInt32